In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, version=1)
mnist.keys()

X, y = mnist['data'], mnist['target']

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

import numpy as np
y = y.astype(np.uint8)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

SGDClassifier(random_state=42)

# 교차 검증을 사용한 정확도 측정
가끔 사이킷런이 제공하는 것보다<br>
교차 검증 과정을 더 많이 제어해야 할 필요가 있습니다.

이 때는 교차 검증 기능을 직접 구현하면 됩니다.

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

In [4]:
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.1005
0.10405
0.0993


- sklearn.model_selection.StratifiedKFold: This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
- sklearn.base.clone: returns a new estimator with the same parameters that has not been fitted on any data.

그런데 더 사이킷런이 제공하는 정도로만 제어를 하면 된다면<br>
그냥 제공하는 걸 쓰는 게 더 좋은 방법이라 할 수 있겠습니다.

In [5]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')

array([0.95035, 0.96035, 0.9604 ])

모든 교차 검증 폴드에 대해 정확도가 95% 이상인 것을 볼 수 있습니다.

이게 되게 좋아보일 수 있는데,<br>
모든 이미지를 '5 아님'클래스로 분류하는 더미 분류기를 만들어서 비교해보겠습니다.

In [6]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [7]:
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')

array([0.91125, 0.90855, 0.90915])

어떤 이미지인지 보지도 않고 전부 다 '5 아님' 클래스로 분류했는데도<br>
정확도가 90% 이상이 나옵니다.

전체 이미지 중 10% 정도만 5일 것이기 때문에<br>
무조건 '5 아님' 으로 예측하면 정확히 맞출 확률이 90%입니다.

이 예제는 정확도를 분류기를 성능 측정 지표로 선호하지 않는 이유를 보여줍니다.<br>
특히 불균형한 데이터셋을 다룰 때 더욱 그렇습니다.

# 오차 행렬
분류기의 성능을 평가하는 더 좋은 방법은 오차 행렬을 조사하는 것입니다.

이 방법은 클래스 A의 샘플이 클래스 B로 분류된 횟수를 세는 방법입니다.<br>
예를 들어서 분류기가 숫자 5를 3으로 잘못 분류한 횟수를 알고 싶다면<br>
오차 행렬의 5행 3열을 보면 됩니다.

오차 행렬을 만들려면 실제 타깃과 비교할 수 있게 먼저 예측값을 만들어야 합니다.<br>
이 때 cross_val_predict() 함수를 사용할 수 있습니다.

- cross_val_predict: cross_val_score 함수처럼 k-겹 교차 검증을 수행하는데 평가 점수를 반환하지 않고 각 테스트 폴드에서 얻은 예측을 반환합니다.

In [8]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [10]:
y_train_pred.shape

(60000,)